In [2]:
# 모의투자 계좌번호 50095584

In [3]:
import keyring


#종합계좌
keyring.set_password('real_app_key', '박인성', 'PSMovGbOkW9YiMVTmtpJhSuPVmxtQciSLcgQ')
keyring.set_password('real_app_secret', '박인성', 'E54hYNDgbs+yFY2doIWpyV/5Rhy7hwutBZlvlS+8UW3uG2/s1mpEKn4pNNFImMsUeuK/ya7P3QPHKGk+pP5WFrH9cYcX5N7F2Vg4qhsgg1efupe3m3TzoedmzJNiwMyYVOPe92T92KNJRZ6mC3UOJuKyFCaSytQCrPFjnk2713UKuezEwMg=')

#모의계좌
keyring.set_password('mock_app_key', '박인성', 'PSqlCpPd6AhBCSbEkV47ZjZj5GWFRRHlE34s')
keyring.set_password('mock_app_secret', '박인성', 'E54hYNDgbs+yFY2doIWpyV/5Rhy7hwutBZlvlS+8UW3uG2/s1mpEKn4pNNFImMsUeuK/ya7P3QPHKGk+pP5WFrH9cYcX5N7F2Vg4qhsgg1efupe3m3TzoedmzJNiwMyYVOPe92T92KNJRZ6mC3UOJuKyFCaSytQCrPFjnk2713UKuezEwMg=')

In [4]:
import json
import requests
import keyring

#key
app_key = keyring.get_password('mock_app_key', '박인성')
app_secret = keyring.get_password('mock_app_secret', '박인성')

#base url
url_base = "https://openapivts.koreainvestment.com:29443" #모의투자

#information
headers = {"content-type":"application/json"}
path = "oauth2/tokenP"
body = {"grant_type":"client_credentials",
        "appkey":app_key, 
        "appsecret":app_secret}

url = f"{url_base}/{path}"
res = requests.post(url, headers=headers, data=json.dumps(body)) # url에 데이터 전송
access_tocken = res.json()['access_token']


In [5]:
# 해쉬키 발급받기
def hashkey(datas):
  path = "uapi/hashkey"
  url = f"{url_base}/{path}"
  headers = {
    'content-Type' : 'application/json',
    'appKey' : app_key,
    'appSecret' : app_secret,
    }
  res = requests.post(url, headers=headers, data=json.dumps(datas))
  hashkey = res.json()["HASH"]

  return hashkey

In [6]:
#주식 현재가 시세 조회하기
path = "uapi/domestic-stock/v1/quotations/inquire-price"
url = f"{url_base}/{path}"

headers = {"Content-Type":"application/json", 
           "authorization": f"Bearer {access_tocken}",
           "appKey":app_key,
           "appSecret":app_secret,
           "tr_id":"FHKST01010100"}
#tr_id는 거래 id에 해당하며, 원하는 작업마다 tr_id가 다름
params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":"005930"
}

res = requests.get(url, headers=headers, params=params)
res.json()['output']['stck_prpr']

'68400'

In [7]:
#시장가 매수 주문하기
path = "uapi/domestic-stock/v1/trading/order-cash"
url = f"{url_base}/{path}"

data = {
    "CANO": "50095584", # 계좌번호 앞 8 자리
    "ACNT_PRDT_CD": "84", # 계좌번호 뒤 2자리
    "PDNO": "005930", # 종목코드
    "ORD_DVSN": "01", # 주문방법 | 시장가: 01 , 지정가 : 00 
    "ORD_QTY": "10", # 주문 수량
    "ORD_UNPR": "0", #주문 단가(시장가의 경우0)
}

headers = {"Content-Type":"application/json", 
           "authorization":f"Bearer {access_tocken}",
           "appKey":app_key,
           "appSecret":app_secret,
           "tr_id":"VTTC0802U",
           "custtype":"P",
           "hashkey" : hashkey(data)}

res = requests.post(url, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1', 'msg_cd': '40100000', 'msg1': '모의투자 영업일이 아닙니다.'}

In [9]:
#지정가 매수 주문하기
path = "uapi/domestic-stock/v1/trading/order-cash"
url = f"{url_base}/{path}"

data = {
    "CANO": "50095584", # 계좌번호 앞 8 자리
    "ACNT_PRDT_CD": "84", # 계좌번호 뒤 2자리
    "PDNO": "005930", # 종목코드
    "ORD_DVSN": "00", # 주문방법 | 시장가: 01 , 지정가 : 00 
    "ORD_QTY": "10", # 주문 수량
    "ORD_UNPR": "50000", #주문 단가(시장가의 경우0)
}

headers = {"Content-Type":"application/json", 
           "authorization":f"Bearer {access_tocken}",
           "appKey":app_key,
           "appSecret":app_secret,
           "tr_id":"VTTC0802U",
           "custtype":"P",
           "hashkey" : hashkey(data)}

res = requests.post(url, headers=headers, data=json.dumps(data))
res.json()

{'rt_cd': '1', 'msg_cd': '40100000', 'msg1': '모의투자 영업일이 아닙니다.'}

In [11]:
# KRX_FWDG_ORD_ORGNO = res.json()["output"]["KRX_FWDG_ORD_ORGNO"] #한국거래소전송주문조직번호
# ODNO = res.json()["output"]["ODNO"]

#정정주문
# path = "uapi/domestic-stock/v1/trading/order-rvsencl"
# url = f"{url_base}/{path}"

# data ={
#     "CANO": "계좌번호 앞 8자리",
#     "ACNT_PRDT_CD": "계좌번호 뒤 2자리",
#     "KRX_FWDG_ORD_ORGNO":KRX_FWDG_ORD_ORGNO, #한국거래소전송주문조직번호
#     "ORGN_ODNO":ODNO, #주문번호
#     "RVSE_CNCL_DVSN_CD":"03", #주문방법 | 최유리 지정가 03
#     "ORD_DVSN":"01", #정정(취소는 02)
#     "ORD_QTY":"10", #주문수량
#     "ORD_UNPR":"0", #주문 단가 (시장가의 경우 0)
#     "QTY_ALL_ORD_YN": "Y", #잔량 전부(잔량 일부는 N)
# }

# headers = {"Content-Type":"application/json", 
#           "authorization":f"Bearer {access_tocken}",
#           "appKey": app_key,
#           "appSecret":app_secret,
#           "tr_id":"VTTC0803U",
#           "custtype":"P", 
#           "hashkey" : hashkey(data)}

# res = requests.post(url, headers=headers, data=json.dumps(data))
# res.json() #res.text

In [12]:
#시장가 매도 주문하기
# path = "uapi/domestic-stock/v1/trading/order-cash"
# url = f"{url_base}/{path}"

# data = {
#     "CANO": "50095584", # 계좌번호 앞 8 자리
#     "ACNT_PRDT_CD": "84", # 계좌번호 뒤 2자리
#     "PDNO": "005930", # 종목코드
#     "ORD_DVSN": "01", # 주문방법 | 시장가: 01 , 지정가 : 00 
#     "ORD_QTY": "10", # 주문 수량
#     "ORD_UNPR": "0", #주문 단가(시장가의 경우0)
# }

# headers = {"Content-Type":"application/json", 
#            "authorization":f"Bearer {access_tocken}",
#            "appKey":app_key,
#            "appSecret":app_secret,
#            "tr_id":"VTTC0801U",
#            "custtype":"P",
#            "hashkey" : hashkey(data)}

# res = requests.post(url, headers=headers, data=json.dumps(data))
# res.json()

In [13]:
#주식 잔고조회
path = "/uapi/domestic-stock/v1/trading/inquire-balance"
url = f"{url_base}/{path}"

headers = {"Content-Type":"application/json", 
           "authorization":f"Bearer {access_tocken}",
           "appKey":app_key,
           "appSecret":app_secret,
           "tr_id":"VTTC8434R"}

params = {
    "CANO": "50095584", # 계좌번호 앞 8 자리
    "ACNT_PRDT_CD": "84", # 계좌번호 뒤 2자리
    "AFHR_FLPR_YN": "N", #시간외 당일가 여부
    "OFL_YN": "", # 공란,
    "INQR_DVSN": "01", # 조회구분
    "UNPR_DVSN": "01", # 단가구분
    "FUND_STTL_ICLD_YN": "N", # 펀드 결제분 포함 여부
    "FNCG_AMT_AUTO_RDPT_YN": "N", # 융자금액 자동상환 여부
    "PRCS_DVSN": "00", #처리구분 (00 전일매매포함)
    "CTX_AREA_FK100": "", #연속조회검색조건100
    "CTX_AREA_NK100": "" #연속조회키100
}

res = requests.post(url, headers=headers, params=params)
res.json()


{'ctx_area_fk100': '                                                                                                    ',
 'ctx_area_nk100': '                                                                                                    ',
 'output1': [],
 'output2': [{'dnca_tot_amt': '500000000',
   'nxdy_excc_amt': '500000000',
   'prvs_rcdl_excc_amt': '500000000',
   'cma_evlu_amt': '0',
   'bfdy_buy_amt': '0',
   'thdt_buy_amt': '0',
   'nxdy_auto_rdpt_amt': '0',
   'bfdy_sll_amt': '0',
   'thdt_sll_amt': '0',
   'd2_auto_rdpt_amt': '0',
   'bfdy_tlex_amt': '0',
   'thdt_tlex_amt': '0',
   'tot_loan_amt': '0',
   'scts_evlu_amt': '0',
   'tot_evlu_amt': '500000000',
   'nass_amt': '500000000',
   'fncg_gld_auto_rdpt_yn': '',
   'pchs_amt_smtl_amt': '0',
   'evlu_amt_smtl_amt': '0',
   'evlu_pfls_smtl_amt': '0',
   'tot_stln_slng_chgs': '0',
   'bfdy_tot_asst_evlu_amt': '500000000',
   'asst_icdc_amt': '0',
   'asst_icdc_erng_rt': '0.00000000'}],
 'rt_cd': '0',
 'msg_cd': '203

In [15]:

res.json()['output2']
#보유종목 데이터 프레임으로 변경
#import pandas as pd
#ap = pd.DataFrame.from_records(rs.json()['output1'])

[{'dnca_tot_amt': '500000000',
  'nxdy_excc_amt': '500000000',
  'prvs_rcdl_excc_amt': '500000000',
  'cma_evlu_amt': '0',
  'bfdy_buy_amt': '0',
  'thdt_buy_amt': '0',
  'nxdy_auto_rdpt_amt': '0',
  'bfdy_sll_amt': '0',
  'thdt_sll_amt': '0',
  'd2_auto_rdpt_amt': '0',
  'bfdy_tlex_amt': '0',
  'thdt_tlex_amt': '0',
  'tot_loan_amt': '0',
  'scts_evlu_amt': '0',
  'tot_evlu_amt': '500000000',
  'nass_amt': '500000000',
  'fncg_gld_auto_rdpt_yn': '',
  'pchs_amt_smtl_amt': '0',
  'evlu_amt_smtl_amt': '0',
  'evlu_pfls_smtl_amt': '0',
  'tot_stln_slng_chgs': '0',
  'bfdy_tot_asst_evlu_amt': '500000000',
  'asst_icdc_amt': '0',
  'asst_icdc_erng_rt': '0.00000000'}]